In [1]:
import os
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

os.chdir('./gdrive/MyDrive/Colab Notebooks/cheng_ta/final/dataset')      # 檔案目錄

Mounted at /content/gdrive


In [2]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd
import os
import pickle
import random
import time
import importlib
import sys
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import predictModels as pm

In [3]:
with open('./Attributes/nodeA/transports.pickle', 'rb') as f:
    transports = pickle.load(f)

with open('./Attributes/nodeA/lambdas.pickle', 'rb') as f:
    lambdas = pickle.load(f)
    
with open('./Attributes/nodeA/atomsAdj.pickle', 'rb') as f:
    atomsAdj = pickle.load(f)
    
with open('./Attributes/nodeA/atomAttributes.pickle', 'rb') as f:
    atomAttributes = pickle.load(f)

In [4]:
with open('./dataset/datas.pickle', 'rb') as f:
    datas = pickle.load(f)

# node labels
Nclasses = []
for i in range(len(datas)):
    temp = torch.tensor(datas[i]['nodesLabel'], dtype=torch.long)
    Nclasses.append(temp)

classWeight = [0] * 38
for i in range(len(Nclasses)):
    for j in range(Nclasses[i].shape[0]):
        c = Nclasses[i][j]
        classWeight[c] += 1
classWeight = torch.tensor(classWeight, dtype=torch.float)
classWeight = classWeight / torch.sum(classWeight)
classWeight = 1 / classWeight
classWeight = classWeight / torch.sum(classWeight)
del datas
len(Nclasses), classWeight.shape

(2000, torch.Size([38]))

# node 1 train

In [5]:
# model 參數
inputD = 4
outputD = 38
GCN_layer = 3

# 訓練超參數 train
modelSave = './models/pre2/nodeModel_2.pt'
lossSave = './models/pre2/nodeLoss_2.pickle'
size_batch = 16
epochs = 30
lr = 0.01
weight_decay = 0
shuffle_data = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model = pm.predModel_2(inputD, outputD, GCN_layer)
for i in range(len(atomsAdj)):
    atomsAdj[i] = atomsAdj[i].to(device)
    atomAttributes[i] = atomAttributes[i].to(device)
model.setModel(atomsAdj, atomAttributes)

In [7]:
def train_nodeAttributes(model,
                        transports,
                        lambdas,
                        labels,
                        size_batch: int = 500,
                        epochs: int = 100,
                        lr: float = 1e-1,
                        weight_decay: float = 0,
                        shuffle_data: bool = True,
                        device=None):
    """
    training a FGWF model
    Args:
        model: a FGWF model
        database: a list of data, each element is a list representing [cost, distriubtion, feature, label]
        size_batch: the size of batch, deciding the frequency of backpropagation
        epochs: the number epochs
        lr: learning rate
        weight_decay: the weight of the l2-norm regularization of parameters
        shuffle_data: whether shuffle data in each epoch
        zeta: the weight of the regularizer enhancing the diversity of atoms
        mode: fit or transform
        visualize_prefix: display learning result after each epoch or not
    """
    global modelSave, lossSave, classWeight
    
    if device is not None:
        print(device)
        model.to(device)
        lambdas = lambdas.to(device)
        classWeight = classWeight.to(device)
        for i in range(len(transports)):
            labels[i] = labels[i].to(device)
            for j in range(len(transports[i])):
                transports[i][j] = transports[i][j].to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    lossfun = nn.CrossEntropyLoss(weight=classWeight)
    model.train()

    num_samples = len(transports)
    index_samples = list(range(num_samples))
    index_split = int(num_samples * 0.8)
    
    random.shuffle(index_samples)
    index_train = index_samples[:index_split]
    index_val = index_samples[index_split:]
    
    epoch_metric = {'loss':[], 'accu':[], 'f1':[]}
    
    t_start = time.time()
    
    for epoch in range(epochs):
        batch_loss = []

        if shuffle_data:
            random.shuffle(index_train)
        
        total_loss = 0.
        counts = 0
        for i in index_train:
            trans, lamb = transports[i], lambdas[i, :]
            y = model(trans, lamb)
            l = labels[i]
            loss = lossfun(y, l)
            total_loss += loss
            
            counts += 1
            if counts % size_batch == 0 or counts == num_samples:
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
                batch_loss.append(total_loss.item())
                total_loss = 0.
                counts = 0
            
        el = sum(batch_loss) / len(batch_loss)
        epoch_metric['loss'].append(el)
        
        preds = []
        trues = []
        for i in index_val:
            with torch.no_grad():
                trans, lamb = transports[i], lambdas[i, :]
                y = model(trans, lamb)
            pred = torch.argmax(y, dim=1).cpu().detach()
            true = labels[i].cpu().detach()
            preds.append(pred)
            trues.append(true)
        preds = torch.cat(preds, dim=0)
        trues = torch.cat(trues, dim=0)
            
        accu = accuracy_score(preds, trues)
        f1 = f1_score(preds, trues, average='macro')
        epoch_metric['accu'].append(accu)
        epoch_metric['f1'].append(f1)
        print('epoch loss: {}, accu/f1:{}/{}, epoch:{}/{}, time:{}'.format(el, accu, f1, epoch, epochs, time.time()-t_start))
        
        with open(lossSave, 'wb') as f:
              pickle.dump(epoch_metric, f)
        torch.save(model.state_dict(), modelSave)

In [8]:
train_nodeAttributes(model=model,
                      transports=transports,
                      lambdas=lambdas,
                      labels=Nclasses,
                      size_batch=size_batch,
                      epochs=epochs,
                      lr=lr,
                      weight_decay=weight_decay,
                      shuffle_data=shuffle_data,
                      device=device
                      )

cuda
epoch loss: 54.3127082824707, accu/f1:0.7320200125078173/0.08830424258074339, epoch:0/30, time:1749.3737325668335


In [ ]:
with open('./models/pre2/nodeLoss_2.pickle', 'rb') as f:
    graphLoss_2 = pickle.load(f)

In [ ]:
graphLoss_2

{'loss': [36.3759880065918], 'accu': [0.046875], 'f1': [0.023529411764705882]}